# Rats Birthday Mixtape Synthesizer 🐀🎶

Sean Wu (sywu@duck.com), July 17 2022

## Project Motivation
The dream is to send unique happy birthday audios that appears way more high effort than a simple "omg happy birthday sean!🎊🍰" message.

The song "Rats Birthday Remix" is stuck in my head. The song sings to a character named "Michael" from the Rats Movie 2. Here we will generalize the song by synthesizing the song with any song derived from the user's input. It's funny, catchy, and most importantly, automatic.

## Past Work
Something similar does exists on this cool website https://itsyourbirthday.today/?name=sean, but I'm unimpressed by how they overlap the audio. We can do better!

## Design Parameters
| Functions      | Objectives | Constraints     |
|    :----:   |    :----:   |    :----:   |
| Create an audio birthday message      | Be fast/convenient       | Use the Rats Mixtape backing track|
|  Embed a new name  | Be fun        |     |
|    | Be smooth/original        |      |

## Data Prep
First let's pull the "Rats Birthday Remix" from YouTube. This version has already conveniently cleaned out "Michael."

In [11]:
# modules pull mp3/mp4 data from YouTube and save to system
from pytube import YouTube
import os

birthdaymix = "https://www.youtube.com/watch?v=hSh_5qmfQLw"
yt = YouTube(birthdaymix)
video = yt.streams.filter(only_audio=True).first()
destination = '.'

birthday_file = 'RatsBirthdayMixtape[withoutMichael].mp3'

# check if file already exists
if not os.path.exists(os.path.join(destination, birthday_file)):
    # download the file
    out_file = video.download(output_path=destination)
    os.rename(out_file, birthday_file)
    
    # result of success
    print(birthday_file + " has been successfully downloaded.")
else:
    print(birthday_file + " has already been downloaded in this directory.")

RatsBirthdayMixtape[withoutMichael].mp3 has already been downloaded in this directory.


## Create New "Michael"

In [17]:
# modules python Text To Speech and to play audio
import pyttsx3
import IPython.display as ipd

engine = pyttsx3.init()
voices = engine.getProperty("voices")
engine.setProperty("voice", voices[0].id)
print("Who's birthday? [No spaces. For Jlo or JT, enter J-Lo or J-T]")

birthday_name = str(input(">>")) or 'Michael'
name_file = birthday_name + ".mp3"

if not os.path.exists(os.path.join('.', name_file)):
    engine.save_to_file(birthday_name, name_file)
    engine.runAndWait()

ipd.Audio(name_file)

Who's birthday? [No spaces. For Jlo or JT, enter J-Lo or J-T]
>>J-Lo


## Overlay New "Michael" into Rats Mixtape
Overlay requires files to be in .wav format. First we convert, then we overlay.

In [18]:
%%capture
# capture suppresses verbose output

# convert mp3 files to wav files
name_wav = birthday_name + '.wav'
%sx ffmpeg -i $name_file $name_wav

birthday_wav = 'RatsBirthdayMixtape[withoutMichael].wav'
%sx ffmpeg -i $birthday_file $birthday_wav

In [19]:
from pydub import AudioSegment
birthday_audio = AudioSegment.from_file(birthday_wav, format="wav")
name_audio = AudioSegment.from_file(name_wav, format="wav")

# sound1 6 dB louder
louder = name_audio + 6

# Overlay sound2 over sound1 at position 0  (use louder instead of sound1 to use the louder version)
overlayiter = birthday_audio.overlay(louder, position=8300)
timestamps = [int(1000 * ts_sec) for ts_sec in [12.4,28.85,33.0,37.1,53.5,57.6,61.8,94.5,98.6,102.75,127.35,131.4,135.5,143.7,147.9,151.9,160.1,164.3,168.4,176.5,180.9]]
for ts_millisec in timestamps:
    overlayiter = overlayiter.overlay(louder, position=ts_millisec)

# simple export
output = "RatsBirthdayMixtape[" + birthday_name +"].mp3"
file_handle = overlayiter.export(output, format="mp3")
ipd.Audio(output)

## Clean Up
Deletes all the intermediary audio files to keep the directory clean

In [20]:
for file in [name_file, name_wav]:
    if os.path.exists(os.path.join('.', file)):
        os.remove(file)